In [98]:
import numpy as np
import pandas as pd

In [99]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [100]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [101]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [102]:
from sklearn.metrics import f1_score

In [103]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

In [104]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [105]:
import string 
import nltk   

In [106]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

stop_words[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gafed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [107]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [108]:
word_tokenizer = nltk.WordPunctTokenizer()

In [109]:
def process_data(data):
    texts = []
    tokens  = word_tokenizer.tokenize(data)
    tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words)]
    texts.append(tokens)
    return texts

In [110]:
for i in range (0, len(X_test)):
    X_test[i] = X_test[i].lower()
    X_test[i] = "".join(l for l in X_test[i] if l not in string.punctuation) 
    
X_test

array(['шестой кассационный суд в самаре начнет работу в разных зданиях – фото – коммерсантъ',
       'что такое индексация алиментов кем и в каких случаях производится каковы порядок и правила процедуры',
       'женщинам  империя меха  part 12', ...,
       'администрация лесного района тверской области  75я годовщина снятия блокады',
       'сонник изменение сознания к чему снится изменение сознания видеть во сне  сонник дома солнца',
       'строительство базы команды грандрапидс гриффинс грандрапидс сша'],
      dtype=object)

In [111]:
for i in range (0, len(X_train)):
    X_train[i] = X_train[i].lower()
    X_train[i] = "".join(l for l in X_train[i] if l not in string.punctuation) 
    
X_train

array(['эксминистр экономики молдовы  главе мидэи цель которого сделать из республики не просителя а донора надо избегать долгого нахождения н�',
       'эта песня стала известна многим телезрителям благодаря сериалу диверсант2',
       'банши 4 сезон 2 серия бремя красоты смотреть онлайн', ...,
       '1941 plymouth special deluxe hot rod automaticsmall blockpower steering ac for sale photos technical specifications description',
       'купить its skin сыворотка питательная power 10 formula ye effector 30 мл в москве  wowcream',
       'технический спирт в канистрах и флаконах купить в москве'],
      dtype=object)

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV

vectorizer = TfidfVectorizer()

In [113]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [114]:
X_train_vectorized

<135309x202653 sparse matrix of type '<class 'numpy.float64'>'
	with 1144319 stored elements in Compressed Sparse Row format>

In [115]:
feature_names = np.array(vectorizer.get_feature_names())

In [116]:
id_ = 42

print(X_train[id_])

x_vector = X_train_vectorized.getrow(id_).toarray()[0]

[feature for feature in feature_names[x_vector > 0]]

презентация на тему приближенное значение абсолютная и относительная погрешнос


['абсолютная',
 'значение',
 'на',
 'относительная',
 'погрешнос',
 'презентация',
 'приближенное',
 'тему']

In [117]:
%%time

model = LogisticRegressionCV(cv=5, random_state=0, max_iter=2000).fit(X_train_vectorized, y_train)

Wall time: 7min 45s


In [118]:
%%time

y_pred = model.predict(X_train_vectorized)

Wall time: 9 ms


In [119]:
f1_score(y_train, y_pred)

0.9953823088455771

In [120]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
